In [1]:
import sapien.core as sapien
import numpy as np
from PIL import Image, ImageColor
import open3d as o3d
from sapien.utils.viewer import Viewer
from transforms3d.euler import mat2euler


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
opencv-contrib not installed, some features will be disabled.
Please install with `pip3 install opencv-contrib-python`


In [3]:
engine = sapien.Engine()
renderer = sapien.VulkanRenderer()
engine.set_renderer(renderer)

scene = engine.create_scene()
scene.set_timestep(1 / 100.0)


scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5], shadow=True)
scene.add_point_light([1, 2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([1, -2, 2], [1, 1, 1], shadow=True)
scene.add_point_light([-1, 0, 1], [1, 1, 1], shadow=True)

# ---------------------------------------------------------------------------- #
# Camera
# ---------------------------------------------------------------------------- #
near, far = 0.1, 100
width, height = 640, 480
camera_mount_actor = scene.create_actor_builder().build_kinematic()
camera = scene.add_mounted_camera(
    name="camera",
    actor=camera_mount_actor,
    pose=sapien.Pose(),  # relative to the mounted actor
    width=width,
    height=height,
    fovy=np.deg2rad(35),
    near=near,
    far=far,
)

print('Intrinsic matrix\n', camera.get_camera_matrix())

# Compute the camera pose by specifying forward(x), left(y) and up(z)
cam_pos = np.array([-2, -2, 3])
forward = -cam_pos / np.linalg.norm(cam_pos)
left = np.cross([0, 0, 1], forward)
left = left / np.linalg.norm(left)
up = np.cross(forward, left)
mat44 = np.eye(4)
mat44[:3, :3] = np.stack([forward, left, up], axis=1)
mat44[:3, 3] = cam_pos
camera_mount_actor.set_pose(sapien.Pose.from_transformation_matrix(mat44))

scene.step()  # make everything set
scene.update_render()
camera.take_picture()

# ---------------------------------------------------------------------------- #
# RGBA
# ---------------------------------------------------------------------------- #
rgba = camera.get_float_texture('Color')  # [H, W, 4]
# An alias is also provided
# rgba = camera.get_color_rgba()  # [H, W, 4]
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_pil = Image.fromarray(rgba_img)
rgba_pil.save('color.png')

# ---------------------------------------------------------------------------- #
# XYZ position in the camera space
# ---------------------------------------------------------------------------- #
# Each pixel is (x, y, z, is_valid) in camera space (OpenGL/Blender)
position = camera.get_float_texture('Position')  # [H, W, 4]

# OpenGL/Blender: y up and -z forward
points_opengl = position[..., :3][position[..., 3] > 0]
points_color = rgba[position[..., 3] > 0][..., :3]
# Model matrix is the transformation from OpenGL camera space to SAPIEN world space
# camera.get_model_matrix() must be called after scene.update_render()!
model_matrix = camera.get_model_matrix()
points_world = points_opengl @ model_matrix[:3, :3].T + model_matrix[:3, 3]

# SAPIEN CAMERA: z up and x forward
# points_camera = points_opengl[..., [2, 0, 1]] * [-1, -1, 1]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_world)
pcd.colors = o3d.utility.Vector3dVector(points_color)
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame()


[2022-06-08 14:12:03.176] [SAPIEN] [warning] A second engine will shared the same internal structures with the first one. Arguments passed to constructor will be ignored.
[2022-06-08 14:12:03.176] [SAPIEN] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.
[2022-06-08 14:12:03.176] [SAPIEN] [warning] Setting renderer more than once should be avoided.


Intrinsic matrix
 [[761.18274   0.      320.        0.     ]
 [  0.      761.18274 240.        0.     ]
 [  0.        0.        1.        0.     ]
 [  0.        0.        0.        1.     ]]


In [12]:
T_world_vulkan = model_matrix
T_world_vulkan.round(2)

array([[ 0.71,  0.51, -0.49, -2.  ],
       [-0.71,  0.51, -0.49, -2.  ],
       [ 0.  ,  0.69,  0.73,  3.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32)

In [8]:
T_vulkan_opencv = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

In [9]:
T_world_opencv = T_world_vulkan @ T_vulkan_opencv

In [11]:
T_world_opencv.round(2)

array([[ 0.71, -0.51,  0.49, -2.  ],
       [-0.71, -0.51,  0.49, -2.  ],
       [ 0.  , -0.69, -0.73,  3.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]])

In [15]:
camera.get_pose().to_transformation_matrix().round(2)

array([[ 0.49, -0.71,  0.51, -2.  ],
       [ 0.49,  0.71,  0.51, -2.  ],
       [-0.73, -0.  ,  0.69,  3.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32)

In [19]:
(
    np.linalg.inv(
    camera.get_pose().to_transformation_matrix().round(2)
) @ T_world_opencv
).round(2)

array([[ 0.  , -0.01,  0.99,  0.  ],
       [-1.  , -0.  , -0.  ,  0.  ],
       [ 0.  , -1.  , -0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ]])